In [230]:
#These are the libraries you can use.  You may add any libraries directy related to threading if this is a direction
#you wish to go (this is not from the course, so it's entirely on you if you wish to use threading).  Any
#further libraries you wish to use you must email me, james@uwaterloo.ca, for permission.

from IPython.display import display, Math, Latex

import pandas as pd
import numpy as np
import numpy_financial as npf
import yfinance as yf
import matplotlib.pyplot as plt
import random
from datetime import datetime
import cvxpy as cp

## Group Assignment
### Team Number: 08
### Team Member Names: Abbas, Gurmehar, Elaine
### Team Strategy Chosen: Market Meet

Disclose any use of AI for this assignment below (detail where and how you used it).  Please see the course outline for acceptable uses of AI.


In [ ]:
tickers_file_name = "Tickers_Example.csv"
tickers_file_original = pd.read_csv(tickers_file_name , header = None, names=['Ticker'])

start_date_clean = "2023-10-01"
end_date_clean = "2024-09-30"
drop_tickers = []

for tick in tickers_file_original['Ticker']:    

    try:

        ticker = yf.Ticker(tick) 

        if ticker.info['quoteType'] != 'EQUITY':
            drop_tickers.append(tick)
            continue
                
        if ticker.fast_info.get('currency') not in ['USD', 'CAD']:
            drop_tickers.append(tick)
            continue

        history = ticker.history(start = start_date_clean, end = end_date_clean)

        trading_days = history['Volume'].resample('ME').count()
        volume = history['Volume'].resample('ME').sum()
        trading_days = trading_days[trading_days>=18]
        volume = volume[volume.index==trading_days.index]
        monthly_volume = volume.mean()

        if monthly_volume<100000: 
            drop_tickers.append(tick)

    except:
            drop_tickers.append(tick)
            pass
    
tickers_file_original.drop(tickers_file_original[tickers_file_original["Ticker"].isin(drop_tickers)].index, inplace=True)
tickers_file_original = tickers_file_original.drop_duplicates(subset=['Ticker'], keep='last')
tickers_file_original = tickers_file_original.reset_index(drop=True)
tickers_file_original.index = tickers_file_original.index + 1

$NVAC: possibly delisted; no price data found  (period=5d)


In [232]:
# take exchange rates from yfinance
start_date = "2023-04-02"
end_date = "2024-09-30"

exchange = 'USDCAD=X'
exchange_ticker = yf.Ticker(exchange)
exchange_rate = exchange_ticker.history(interval = '1mo', start=start_date, end=end_date)["Close"]
exchange_rate = pd.DataFrame(exchange_rate)
exchange_rate.index = exchange_rate.index.date
exchange_rate_value = exchange_rate.mean().iloc[0]

In [233]:
sp500 = yf.Ticker("^GSPC").history(start=start_date, end=end_date)['Close'].mul(exchange_rate_value)
sp500.index = sp500.index.date
sp500_return = sp500.pct_change().dropna()


tsx60 = yf.Ticker("XIU.TO").history(start=start_date, end=end_date)['Close']
tsx60.index = tsx60.index.date
tsx60_return = tsx60.pct_change().dropna()

total_investment = 1000000

sp500val = (total_investment / sp500.iloc[0]) * sp500.iloc[-1]
sp500val = round(sp500val,2)

tsx60val = (total_investment / tsx60.iloc[0]) * tsx60.iloc[-1]
tsx60val = round(tsx60val,2)

average = (sp500val + tsx60val) / 2
average = round(average,2)

common_dates = sp500_return.index.intersection(tsx60_return.index)

average_df = pd.DataFrame(data = {"% Return" : (sp500_return.loc[common_dates] + tsx60_return.loc[common_dates])/2}, index = common_dates)


In [234]:
stocks = {key: 0 for key in tickers_file_original['Ticker'].values}
current_data = pd.DataFrame()
metrics = pd.DataFrame(index=list(stocks.keys()), columns=['SP-beta','SP-corr','TSX-beta','TSX-corr', 'Volatility', 'S&P500', 'TSX60'])

url = "https://en.wikipedia.org/wiki/List_of_S%26P_500_companies"
sp500_df = pd.read_html(url)[0]
sp500companies = sp500_df.Symbol.tolist()

tsx60companies = ["RY.TO", "TD.TO", "SHOP.TO", "ENB.TO", "CNR.TO", "SU.TO", "BN.TO", "CSU.TO", 
    "MFC.TO", "TRP.TO", "BMO.TO", "BNS.TO", "ATD.TO", "ABX.TO", "BCE.TO", "CM.TO", 
    "CP.TO", "NTR.TO", "TRI.TO", "WCN.TO", "MG.TO", "L.TO", "IFC.TO", "WN.TO", 
    "SLF.TO", "FNV.TO", "TECK.B.TO", "DOL.TO", "OTEX.TO", "FM.TO", "IMO.TO", 
    "CVE.TO", "PPL.TO", "CNQ.TO", "FTS.TO", "H.TO", "RCI.B.TO", "T.TO", "AQN.TO", 
    "SAP.TO", "MRU.TO", "QSR.TO", "WPM.TO", "POW.TO", "EMA.TO", "AEM.TO", 
    "GIL.TO", "GIB.A.TO", "TOU.TO", "BEP.UN.TO", "BIP.UN.TO", "CTC.A.TO", 
    "CCL.B.TO", "K.TO", "WSP.TO", "IFP.TO"]

def in_sp500(ticker, sp500companies):
    if ticker in sp500companies:
        return 2
    else:
        return 0
    
def in_tsx60(ticker, tsx60companies):
    if ticker in tsx60companies:
        return 2
    else:
        return 0

    
sp500_returns = sp500.pct_change(fill_method=None)
tsx60_returns = tsx60.pct_change(fill_method=None)

for i in stocks:
    try:
        current_data[i] = yf.Ticker(i).history(start=start_date, end=end_date)["Close"]
        volatility = current_data[i].std(axis=0)
        metrics.loc[i, 'Volatility'] = volatility
        metrics.loc[i, 'S&P500'] = in_sp500(i, sp500companies)
        metrics.loc[i, 'TSX60'] = in_tsx60(i, tsx60companies)
        
        stock_returns = current_data[i].pct_change().dropna()
        stock_returns.index = stock_returns.index.date
        
        common_dates = stock_returns.index.intersection(sp500_returns.index).intersection(tsx60_returns.index)

        stock_returns = stock_returns.loc[common_dates]
        aligned_sp500_returns = sp500_returns.loc[common_dates]
        aligned_tsx60_returns = tsx60_returns.loc[common_dates]
        
        covariance = np.cov(stock_returns, aligned_sp500_returns)[0, 1]
        variance = np.var(aligned_sp500_returns)
        beta_sp500 = covariance / variance
        
        correlation_sp500 = np.corrcoef(stock_returns, aligned_sp500_returns)[0, 1]
        
        covariance_tsx = np.cov(stock_returns, aligned_tsx60_returns)[0, 1]
        variance_tsx = np.var(aligned_tsx60_returns)
        beta_tsx60 = covariance_tsx / variance_tsx
        
        correlation_tsx60 = np.corrcoef(stock_returns, aligned_tsx60_returns)[0, 1]
        
        metrics.loc[i, 'SP-corr'] = correlation_sp500
        metrics.loc[i, 'SP-beta'] = beta_sp500
        metrics.loc[i, 'TSX-corr'] = correlation_tsx60
        metrics.loc[i, 'TSX-beta'] = beta_tsx60

        stocks[i] = round(5 * (2 * correlation_sp500 + 0.6 * (abs (1 - beta_sp500)) + 0.6 * (abs (1 - beta_tsx60)) + 2 * correlation_tsx60) 
                        + metrics.loc[i, 'S&P500'] + metrics.loc[i, 'TSX60'] - (0.05 * volatility),2)
    except:
        continue

C:\Users\abbas\AppData\Local\Temp\ipykernel_36964\2334443901.py:42: FutureWarning: The default fill_method='pad' in Series.pct_change is deprecated and will be removed in a future version. Either fill in any non-leading NA values prior to calling pct_change or specify 'fill_method=None' to not fill NA values.
  stock_returns = current_data[i].pct_change().dropna()
C:\Users\abbas\AppData\Local\Temp\ipykernel_36964\2334443901.py:42: FutureWarning: The default fill_method='pad' in Series.pct_change is deprecated and will be removed in a future version. Either fill in any non-leading NA values prior to calling pct_change or specify 'fill_method=None' to not fill NA values.
  stock_returns = current_data[i].pct_change().dropna()
C:\Users\abbas\AppData\Local\Temp\ipykernel_36964\2334443901.py:42: FutureWarning: The default fill_method='pad' in Series.pct_change is deprecated and will be removed in a future version. Either fill in any non-leading NA values prior to calling pct_change or speci

In [235]:
metrics

,SP-beta,SP-corr,TSX-beta,TSX-corr,Volatility,S&P500,TSX60
ABBV,0.127697,0.07765,0.226176,0.128768,19.079447,2,0
ABT,0.233177,0.146113,0.335355,0.196748,6.39828,2,0
ACN,0.788511,0.42553,0.599283,0.302799,27.377846,2,0
AIG,0.783387,0.450687,0.849136,0.45738,8.973511,2,0
AMZN,1.489155,0.608426,0.914082,0.349667,27.388276,2,0
AXP,1.091674,0.572638,1.123309,0.55168,38.141135,2,0
BA,0.918956,0.354122,0.939501,0.338966,23.857705,2,0
BAC,0.952917,0.479334,1.301092,0.612762,5.177568,2,0
BB.TO,1.409429,0.306048,1.448765,0.294541,1.371217,0,0
BIIB,0.623275,0.307328,0.674787,0.311522,32.580907,2,0


In [236]:
stocks = dict(sorted(stocks.items(), key=lambda item: item[1], reverse=True))
stocks = list(stocks.items())[:24]
stocks = dict(stocks)

In [237]:
stocks

{'SHOP.TO': np.float64(16.58),
 'QCOM': np.float64(14.7),
 'RY.TO': np.float64(14.64),
 'USB': np.float64(14.57),
 'TXN': np.float64(14.21),
 'PYPL': np.float64(13.79),
 'TD.TO': np.float64(13.76),
 'BAC': np.float64(13.71),
 'C': np.float64(12.99),
 'BK': np.float64(12.6),
 'CAT': np.float64(12.13),
 'AXP': np.float64(11.98),
 'AMZN': np.float64(11.94),
 'AIG': np.float64(11.73),
 'T.TO': np.float64(11.44),
 'AAPL': np.float64(11.28),
 'BLK': np.float64(11.11),
 'UNP': np.float64(11.03),
 'PM': np.float64(10.89),
 'MO': np.float64(10.5),
 'KO': np.float64(10.36),
 'UPS': np.float64(10.1),
 'ACN': np.float64(9.75),
 'ABT': np.float64(9.4)}

In [238]:
def give_weight(num_stocks):
    stocks_now = list(stocks.keys())[:num_stocks]
    prices = np.array(current_data[stocks_now].iloc[0])
    last_values = np.array(current_data[stocks_now].iloc[-1])
    w = cp.Variable(num_stocks)
    my_value = cp.sum(cp.multiply(last_values, w * 1000000 / prices))
    min_weight = (100 / (2 * num_stocks)) / 100
    objective = cp.Minimize(my_value - sp500val)
    constraints = [
        cp.sum(w) == 1,           
        w >= min_weight,                    
        w <= 0.15                      
    ]
    problem = cp.Problem(objective, constraints)
    problem.solve()
    return w.value

In [239]:
def get_value(weights,i):
    last_row = current_data[list(stocks.keys())[:i]].iloc[-1]
    first_row = current_data[list(stocks.keys())[:i]].iloc[0]
    return np.sum(np.multiply(last_row,np.divide(weights*1000000,first_row)))

In [240]:
def stocks_ith_portfolio(numStocks):
    weight_now = give_weight(numStocks)
    curr_val = get_value(weight_now, numStocks)
    return curr_val, weight_now

In [ ]:
diff = average
close = 0
best_weights = []
best = 0
for i in range(12,25):
    curr_val, weight_now = stocks_ith_portfolio(i)
    if (abs(average - curr_val) < diff):
        diff = abs(curr_val - average)
        best = i
        best_weights = weight_now
        close = curr_val

233.6469075002242
0.0001779427522370657


In [242]:
ranked_stocks = stocks

stockslist = list(ranked_stocks.keys())
weightlist = best_weights
weights = dict(zip(stockslist, weightlist))
total_money = 1000000
final_allocation = {}
for stock, weight in weights.items():
    try:
        if weight > 0:
            allocation = total_money * weight  
            ticker = yf.Ticker(stock)
            
            if ticker.fast_info['currency'] == 'USD':
                price = ticker.history(start="2024-11-22", end="2024-11-23")['Close'].iloc[0] * exchange_rate_value         
            else:
                price = ticker.history(start="2024-11-22", end="2024-11-23")['Close'].iloc[0]          

            num_shares = (allocation / price)
            # considering transaction costs
            if num_shares > 3950:
                num_shares = (allocation - 3.95)/price
                transaction_cost = 3.95
            else:
                num_shares = allocation/(price+0.001)
                transaction_cost = num_shares*0.001
            total_cost = num_shares * price
            
            final_allocation[stock] = {
                "Number of Shares": num_shares,
                "Transaction Cost": transaction_cost,
                "Total Money Spent": total_cost + transaction_cost,
                "Cumulative Transaction Cost": np.round(sum(stock["Transaction Cost"] for stock in final_allocation.values()), 2),
                "Cumulative Total": np.round(total_cost + transaction_cost+ sum(stock["Total Money Spent"] for stock in final_allocation.values()), 2)
            }
    except:
        continue
final = pd.DataFrame(final_allocation)

In [243]:
# put everything into the final portfolio

Portfolio_Final = pd.DataFrame({"Tickers": [], 
                   "Price":[], 
                   "Currency":[], 
                   "Shares":[], 
                   "Value":[], 
                   "Weight":[]})
cumulative_value = 0
cumulative_transaction_cost = 0
for stock, weight in weights.items():
    try:
        if weight > 0:
            allocation = total_money * weight  
            ticker = yf.Ticker(stock)
            if ticker.fast_info.get('currency') == 'USD':
                price = ticker.history(start="2024-11-22", end="2024-11-23")['Close'].iloc[0] * exchange_rate_value         
            else:
                price = ticker.history(start="2024-11-22", end="2024-11-23")['Close'].iloc[0]          
                
            num_shares = (allocation / price)
            
            # considering transaction costs
            if num_shares > 3950:
                num_shares = (allocation - 3.95)/price
                transaction_cost = 3.95
            else:
                num_shares = allocation/(price+0.001)
                transaction_cost = num_shares*0.001
            total_cost = num_shares * price

            value = price * num_shares
            cumulative_transaction_cost += transaction_cost

            new_row = {"Tickers": stock, 
                    "Price": price, 
                    "Currency": "CAD", 
                    "Shares": num_shares, 
                    "Value": value, 
                    "Weight": weight}
            Portfolio_Final.loc[len(Portfolio_Final)] = new_row 
    except:
        continue

Portfolio_Final.index = range(1, len(Portfolio_Final) + 1)

In [244]:
print(f"The Total Value of the Portfolio is {Portfolio_Final['Value'].sum(axis=0)}")
print(f"The Total Transaction Costs of the Portfolio is {cumulative_transaction_cost}")
print(f"The Sum of the weights of each stocks adds up to {(Portfolio_Final['Weight'].sum(axis=0) * 100):,.2f} %")

The Total Value of the Portfolio is 999991.3920923304
The Total Transaction Costs of the Portfolio is 8.607907654410843
The Sum of the weights of each stocks adds up to 100.00 %


In [245]:
Portfolio_Final

,Tickers,Price,Currency,Shares,Value,Weight
1,SHOP.TO,149.479996,CAD,278.742231,41666.387545,0.041667
2,QCOM,212.417609,CAD,196.153567,41666.471542,0.041667
3,RY.TO,174.710007,CAD,620.071668,108332.725201,0.108333
4,USB,71.112961,CAD,2109.290285,149997.877103,0.150000
5,TXN,268.248538,CAD,559.180833,149999.440966,0.150000
6,PYPL,117.555176,CAD,1275.985713,149998.724485,0.150000
7,TD.TO,78.510002,CAD,1910.560256,149998.089768,0.150000
8,BAC,63.675158,CAD,654.352715,41666.012545,0.041667
9,C,94.618570,CAD,440.359935,41666.227422,0.041667
10,BK,108.572918,CAD,383.763129,41666.282528,0.041667


In [246]:
Stocks_Final = pd.DataFrame({"Tickers": Portfolio_Final['Tickers'], 
                             "Shares": Portfolio_Final['Shares']})

Stocks_Final.to_csv('Stocks_Group_08.csv', index=False)


## Contribution Declaration

The following team members made a meaningful contribution to this assignment:

Abbas, Gurmehar, Elaine